In [1]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import pandas as pd
import copy

from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, ConcatDataset
from rdkit import Chem
from conf_ensemble_dataset_in_memory import ConfEnsembleDataset
from rdkit.Chem import AllChem #needed for rdForceFieldHelpers
from collections import defaultdict
from litschnet import LitSchNet
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from rmsd_predictor_evaluator import RMSDPredictorEvaluator
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

In [2]:
pl.seed_everything(42, workers=True)

Global seed set to 42


42

# Data preparation

In [3]:
data_dir = 'data/'

In [4]:
# run once to preprocess datasets and generate chunks
# dataset = ConfEnsembleDataset()
# dataset = ConfEnsembleDataset(dataset='platinum') # 16G

In [5]:
platinum_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('platinum')]
platinum_n_chunks = len(platinum_chunks)

In [6]:
platinum_datasets = []
for chunk_number in tqdm(range(platinum_n_chunks)) :
    dataset = ConfEnsembleDataset(dataset='platinum', loaded_chunk=chunk_number)
    platinum_datasets.append(dataset)
platinum_dataset = ConcatDataset(platinum_datasets)

100%|█████████████████████████████████████████████| 2/2 [00:08<00:00,  4.04s/it]


In [7]:
pdbbind_chunks = [filename for filename in os.listdir(os.path.join(data_dir, 'processed')) if filename.startswith('pdbbind')]
pdbbind_n_chunks = len(pdbbind_chunks)

In [8]:
for iteration in range(2) :
    with open(os.path.join(data_dir, 'scaffold_splits', f'train_smiles_scaffold_split_{iteration}.txt'), 'r') as f :
        train_smiles = f.readlines()
        train_smiles = [smiles.strip() for smiles in train_smiles]
    
    with open(os.path.join(data_dir, 'scaffold_splits', f'val_smiles_scaffold_split_{iteration}.txt'), 'r') as f :
        val_smiles = f.readlines()
        val_smiles = [smiles.strip() for smiles in val_smiles]

    with open(os.path.join(data_dir, 'scaffold_splits', f'test_smiles_scaffold_split_{iteration}.txt'), 'r') as f :
        test_smiles = f.readlines()
        test_smiles = [smiles.strip() for smiles in test_smiles]
        
    train_datasets = []
    val_datasets = []
    test_datasets = []

    for chunk_number in tqdm(range(pdbbind_n_chunks)) :

        dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                      smiles_list=train_smiles,
                                     filter_out_bioactive=True)
        train_datasets.append(dataset)

        dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                      smiles_list=val_smiles,
                                     filter_out_bioactive=True)
        val_datasets.append(dataset)

        dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                      smiles_list=test_smiles)
        test_datasets.append(dataset)
        
    train_dataset = ConcatDataset(train_datasets)
    val_dataset = ConcatDataset(val_datasets)
    test_dataset = ConcatDataset(test_datasets)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    experiment_name = f'scaffold_split_{iteration}_new_nobio'
    if experiment_name in os.listdir('lightning_logs') :
        checkpoint_name = os.listdir(os.path.join('lightning_logs', experiment_name, 'checkpoints'))[0]
        checkpoint_path = os.path.join('lightning_logs', experiment_name, 'checkpoints', checkpoint_name)
        litschnet = LitSchNet.load_from_checkpoint(checkpoint_path=checkpoint_path)
    else :
        litschnet = LitSchNet()
        logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
        trainer = pl.Trainer(logger=logger, callbacks=[EarlyStopping(monitor="val_loss", patience=5)], gpus=1)
        trainer.fit(litschnet, train_loader, val_loader)
        trainer.test(litschnet, test_loader)
    
    evaluator = RMSDPredictorEvaluator(model=litschnet)
    evaluator.evaluate(test_dataset)
    evaluator.evaluation_report(experiment_name=experiment_name + '_pdbbind')

    evaluator = RMSDPredictorEvaluator(model=litschnet)
    evaluator.evaluate(platinum_dataset)
    evaluator.evaluation_report(experiment_name=experiment_name + '_platinum')

100%|█████████████████████████████████████████████| 3/3 [03:38<00:00, 72.67s/it]


Grouping data by smiles
Starting evaluation


/home/benoit/bioactive_conformation_predictor/rmsd_predictor_evaluator.py:103: RuntimeWarning: invalid value encountered in long_scalars
  accuracy = len(set(bioactive_real_ranks).intersection(set(bioactive_pred_ranks))) / is_bioactive.sum()
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Grouping data by smiles
Starting evaluation


100%|█████████████████████████████████████████████| 3/3 [03:38<00:00, 72.72s/it]


Grouping data by smiles
Starting evaluation


/home/benoit/bioactive_conformation_predictor/rmsd_predictor_evaluator.py:103: RuntimeWarning: invalid value encountered in long_scalars
  accuracy = len(set(bioactive_real_ranks).intersection(set(bioactive_pred_ranks))) / is_bioactive.sum()
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Grouping data by smiles
Starting evaluation


In [10]:
for iteration in range(2) :
    with open(os.path.join(data_dir, 'random_splits', f'train_smiles_random_split_{iteration}.txt'), 'r') as f :
        train_smiles = f.readlines()
        train_smiles = [smiles.strip() for smiles in train_smiles]
    
    with open(os.path.join(data_dir, 'random_splits', f'val_smiles_random_split_{iteration}.txt'), 'r') as f :
        valid_smiles = f.readlines()
        valid_smiles = [smiles.strip() for smiles in valid_smiles]

    with open(os.path.join(data_dir, 'random_splits', f'test_smiles_random_split_{iteration}.txt'), 'r') as f :
        test_smiles = f.readlines()
        test_smiles = [smiles.strip() for smiles in test_smiles]
        
    train_datasets = []
    val_datasets = []
    test_datasets = []

    for chunk_number in tqdm(range(pdbbind_n_chunks)) :

        dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                      smiles_list=train_smiles,
                                     filter_out_bioactive=True)
        train_datasets.append(dataset)

        dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                      smiles_list=val_smiles,
                                     filter_out_bioactive=True)
        val_datasets.append(dataset)

        dataset = ConfEnsembleDataset(loaded_chunk=chunk_number,
                                      smiles_list=test_smiles)
        test_datasets.append(dataset)
        
    train_dataset = ConcatDataset(train_datasets)
    val_dataset = ConcatDataset(val_datasets)
    test_dataset = ConcatDataset(test_datasets)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    experiment_name = f'random_split_{iteration}_new_nobio'
    if experiment_name in os.listdir('lightning_logs') :
        checkpoint_name = os.listdir(os.path.join('lightning_logs', experiment_name, 'checkpoints'))[0]
        checkpoint_path = os.path.join('lightning_logs', experiment_name, 'checkpoints', checkpoint_name)
        litschnet = LitSchNet.load_from_checkpoint(checkpoint_path=checkpoint_path)
    else :
        litschnet = LitSchNet()
        logger = TensorBoardLogger(save_dir=os.getcwd(), version=experiment_name, name="lightning_logs")
        trainer = pl.Trainer(logger=logger, callbacks=[EarlyStopping(monitor="val_loss", patience=5)], gpus=1)
        trainer.fit(litschnet, train_loader, val_loader)
        trainer.test(litschnet, test_loader)
    
    evaluator = RMSDPredictorEvaluator(model=litschnet)
    evaluator.evaluate(test_dataset)
    evaluator.evaluation_report(experiment_name=experiment_name + '_pdbbind')

    evaluator = RMSDPredictorEvaluator(model=litschnet)
    evaluator.evaluate(platinum_dataset)
    evaluator.evaluation_report(experiment_name=experiment_name + '_platinum')

100%|█████████████████████████████████████████████| 3/3 [03:41<00:00, 73.85s/it]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | schnet     | SchNet    | 455 K 
1 | leaky_relu | LeakyReLU | 0     
2 | sigmoid    | Sigmoid   | 0     
-----------------------------------------
455 K     Trainable params
0         Non-trainable params
455 K     Total params
1.823     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.4286777377128601}
--------------------------------------------------------------------------------
Grouping data by smiles
Starting evaluation


/home/benoit/bioactive_conformation_predictor/rmsd_predictor_evaluator.py:103: RuntimeWarning: invalid value encountered in long_scalars
  accuracy = len(set(bioactive_real_ranks).intersection(set(bioactive_pred_ranks))) / is_bioactive.sum()
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/benoit/anaconda3/envs/GeoMol/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Grouping data by smiles
Starting evaluation
